In [1]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [2]:
import pandas as pd

In [14]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';' )

In [15]:
ucis.shape

(61, 4)

In [17]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%y' )

In [18]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [19]:
ucis.dtypes

Fecha                             datetime64[ns]
Camas UCI ocupadas Covid-19                int64
Camas UCI Disponibles COVID 19             int64
Ocupación UCI COVID 19                    object
% Ocupación                              float64
dtype: object

In [20]:
ucis.head()

,Fecha,Camas UCI ocupadas Covid-19,Camas UCI Disponibles COVID 19,Ocupación UCI COVID 19,% Ocupación
0,2020-04-08,106,543,"19,50%",0.195212
1,2020-04-09,136,608,"22,40%",0.223684
2,2020-04-10,150,596,"25,20%",0.251678
3,2020-04-11,148,597,"24,80%",0.247906
4,2020-04-12,178,576,"30,90%",0.309028


In [21]:
ucis.to_csv( './data/uci_bogota.csv', index = False )